# KITTI 3D Object Detection - Data Exploration and Visualization

This notebook provides comprehensive data exploration, visualization, and analysis for the KITTI 3D object detection dataset.

In [ ]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans, DBSCAN
from sklearn.preprocessing import StandardScaler
import torch
from config import cfg
from dataset import KITTIDataset, build_transforms

# Set style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries imported successfully!")

## 1. Dataset Overview

In [ ]:
# Load dataset
dataset = KITTIDataset(
    root=cfg.DATA_ROOT,
    split="training",
    transform=None
)

print(f"Total samples: {len(dataset)}")
print(f"Image directory: {dataset.image_dir}")
print(f"Label directory: {dataset.label_dir}")

## 2. Data Distribution Analysis

In [ ]:
# Collect all 3D box parameters
all_boxes = []
for i in range(len(dataset)):
    _, target, _ = dataset[i]
    if target.sum() != 0:  # Skip empty targets
        all_boxes.append(target.numpy())

all_boxes = np.array(all_boxes)
print(f"Valid samples with Car objects: {len(all_boxes)}")

# Create DataFrame
df = pd.DataFrame(all_boxes, columns=['x', 'y', 'z', 'length', 'width', 'height', 'rotation_y'])
df.head()

In [ ]:
# Statistical summary
print("\n=== Statistical Summary ===")
print(df.describe())

In [ ]:
# Distribution plots
fig, axes = plt.subplots(2, 4, figsize=(20, 10))
fig.suptitle('Distribution of 3D Box Parameters', fontsize=16, fontweight='bold')

columns = ['x', 'y', 'z', 'length', 'width', 'height', 'rotation_y']
for idx, col in enumerate(columns):
    row = idx // 4
    col_idx = idx % 4
    axes[row, col_idx].hist(df[col], bins=50, edgecolor='black', alpha=0.7)
    axes[row, col_idx].set_title(f'{col.capitalize()} Distribution')
    axes[row, col_idx].set_xlabel(col)
    axes[row, col_idx].set_ylabel('Frequency')
    axes[row, col_idx].grid(True, alpha=0.3)

# Remove empty subplot
fig.delaxes(axes[1, 3])
plt.tight_layout()
plt.savefig(os.path.join(cfg.VIS_DIR, 'parameter_distributions.png'), dpi=300, bbox_inches='tight')
plt.show()

## 3. Correlation Analysis

In [ ]:
# Correlation matrix
plt.figure(figsize=(10, 8))
correlation_matrix = df.corr()
sns.heatmap(correlation_matrix, annot=True, fmt='.2f', cmap='coolwarm', 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Correlation Matrix of 3D Box Parameters', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(cfg.VIS_DIR, 'correlation_matrix.png'), dpi=300, bbox_inches='tight')
plt.show()

## 4. Dimensionality Reduction - PCA

In [ ]:
# Standardize data
scaler = StandardScaler()
data_scaled = scaler.fit_transform(df)

# Apply PCA
pca = PCA()
pca_result = pca.fit_transform(data_scaled)

# Explained variance
explained_variance = pca.explained_variance_ratio_
cumulative_variance = np.cumsum(explained_variance)

# Plot explained variance
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

ax1.bar(range(1, len(explained_variance)+1), explained_variance, alpha=0.7, edgecolor='black')
ax1.set_xlabel('Principal Component')
ax1.set_ylabel('Explained Variance Ratio')
ax1.set_title('PCA - Explained Variance by Component')
ax1.grid(True, alpha=0.3)

ax2.plot(range(1, len(cumulative_variance)+1), cumulative_variance, 'bo-', linewidth=2)
ax2.axhline(y=0.95, color='r', linestyle='--', label='95% Variance')
ax2.set_xlabel('Number of Components')
ax2.set_ylabel('Cumulative Explained Variance')
ax2.set_title('PCA - Cumulative Explained Variance')
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(cfg.VIS_DIR, 'pca_variance.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"\nVariance explained by first 3 components: {cumulative_variance[2]:.2%}")

In [ ]:
# 2D PCA visualization
plt.figure(figsize=(10, 8))
scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1], 
                     c=df['z'], cmap='viridis', alpha=0.6, edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, label='Depth (z)')
plt.xlabel(f'PC1 ({explained_variance[0]:.1%} variance)')
plt.ylabel(f'PC2 ({explained_variance[1]:.1%} variance)')
plt.title('PCA - First Two Principal Components (colored by depth)', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(cfg.VIS_DIR, 'pca_2d.png'), dpi=300, bbox_inches='tight')
plt.show()

## 5. Dimensionality Reduction - t-SNE

In [ ]:
# Apply t-SNE (on subset for speed)
subset_size = min(1000, len(data_scaled))
indices = np.random.choice(len(data_scaled), subset_size, replace=False)
data_subset = data_scaled[indices]

tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
tsne_result = tsne.fit_transform(data_subset)

# Visualize t-SNE
plt.figure(figsize=(10, 8))
scatter = plt.scatter(tsne_result[:, 0], tsne_result[:, 1], 
                     c=df.iloc[indices]['z'], cmap='plasma', alpha=0.6, 
                     edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, label='Depth (z)')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.title('t-SNE Visualization (colored by depth)', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(cfg.VIS_DIR, 'tsne_2d.png'), dpi=300, bbox_inches='tight')
plt.show()

## 6. Clustering Analysis - K-Means

In [ ]:
# Elbow method for optimal K
inertias = []
K_range = range(2, 11)

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(data_scaled)
    inertias.append(kmeans.inertia_)

# Plot elbow curve
plt.figure(figsize=(10, 6))
plt.plot(K_range, inertias, 'bo-', linewidth=2, markersize=8)
plt.xlabel('Number of Clusters (K)')
plt.ylabel('Inertia (Within-cluster sum of squares)')
plt.title('K-Means Elbow Method', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(cfg.VIS_DIR, 'kmeans_elbow.png'), dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
# Apply K-Means with optimal K
optimal_k = 4
kmeans = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(data_scaled)

# Visualize clusters in PCA space
plt.figure(figsize=(10, 8))
scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1], 
                     c=clusters, cmap='tab10', alpha=0.6, 
                     edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, label='Cluster')
plt.xlabel(f'PC1 ({explained_variance[0]:.1%} variance)')
plt.ylabel(f'PC2 ({explained_variance[1]:.1%} variance)')
plt.title(f'K-Means Clustering (K={optimal_k}) in PCA Space', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(cfg.VIS_DIR, 'kmeans_clusters.png'), dpi=300, bbox_inches='tight')
plt.show()

# Cluster statistics
df['cluster'] = clusters
print("\n=== Cluster Statistics ===")
print(df.groupby('cluster').mean())

## 7. Clustering Analysis - DBSCAN

In [ ]:
# Apply DBSCAN
dbscan = DBSCAN(eps=0.5, min_samples=5)
dbscan_clusters = dbscan.fit_predict(data_scaled)

n_clusters = len(set(dbscan_clusters)) - (1 if -1 in dbscan_clusters else 0)
n_noise = list(dbscan_clusters).count(-1)

print(f"Number of clusters: {n_clusters}")
print(f"Number of noise points: {n_noise}")

# Visualize DBSCAN clusters
plt.figure(figsize=(10, 8))
scatter = plt.scatter(pca_result[:, 0], pca_result[:, 1], 
                     c=dbscan_clusters, cmap='tab10', alpha=0.6, 
                     edgecolors='black', linewidth=0.5)
plt.colorbar(scatter, label='Cluster (-1 = noise)')
plt.xlabel(f'PC1 ({explained_variance[0]:.1%} variance)')
plt.ylabel(f'PC2 ({explained_variance[1]:.1%} variance)')
plt.title(f'DBSCAN Clustering in PCA Space', fontweight='bold')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(cfg.VIS_DIR, 'dbscan_clusters.png'), dpi=300, bbox_inches='tight')
plt.show()

## 8. Sample Visualization

In [ ]:
# Visualize sample images with 3D boxes
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
fig.suptitle('Sample Images with 3D Bounding Boxes', fontsize=16, fontweight='bold')

sample_indices = np.random.choice(len(dataset), 6, replace=False)

for idx, ax in enumerate(axes.flat):
    img_idx = sample_indices[idx]
    image, target, img_name = dataset[img_idx]
    
    # Convert tensor to numpy
    if isinstance(image, torch.Tensor):
        image = image.numpy().transpose(1, 2, 0)
    
    ax.imshow(image)
    ax.set_title(f'Sample {img_idx}: {img_name}')
    ax.axis('off')
    
    # Add 3D box info as text
    if target.sum() != 0:
        info_text = f"Depth: {target[2]:.1f}m\nSize: {target[3]:.1f}x{target[4]:.1f}x{target[5]:.1f}m"
        ax.text(0.02, 0.98, info_text, transform=ax.transAxes, 
               fontsize=8, verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8))

plt.tight_layout()
plt.savefig(os.path.join(cfg.VIS_DIR, 'sample_images.png'), dpi=300, bbox_inches='tight')
plt.show()

## 9. Feature Engineering Insights

In [ ]:
# Create additional features
df['volume'] = df['length'] * df['width'] * df['height']
df['aspect_ratio'] = df['length'] / df['width']
df['distance_3d'] = np.sqrt(df['x']**2 + df['y']**2 + df['z']**2)

# Visualize new features
fig, axes = plt.subplots(1, 3, figsize=(18, 5))
fig.suptitle('Engineered Features Distribution', fontsize=14, fontweight='bold')

axes[0].hist(df['volume'], bins=50, edgecolor='black', alpha=0.7, color='skyblue')
axes[0].set_title('Volume Distribution')
axes[0].set_xlabel('Volume (m³)')
axes[0].set_ylabel('Frequency')
axes[0].grid(True, alpha=0.3)

axes[1].hist(df['aspect_ratio'], bins=50, edgecolor='black', alpha=0.7, color='lightcoral')
axes[1].set_title('Aspect Ratio Distribution')
axes[1].set_xlabel('Length / Width')
axes[1].set_ylabel('Frequency')
axes[1].grid(True, alpha=0.3)

axes[2].hist(df['distance_3d'], bins=50, edgecolor='black', alpha=0.7, color='lightgreen')
axes[2].set_title('3D Distance Distribution')
axes[2].set_xlabel('Distance (m)')
axes[2].set_ylabel('Frequency')
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(cfg.VIS_DIR, 'engineered_features.png'), dpi=300, bbox_inches='tight')
plt.show()

## 10. Summary Statistics

In [ ]:
# Generate comprehensive summary
summary = {
    'Total Samples': len(dataset),
    'Valid Car Objects': len(all_boxes),
    'Average Depth (z)': df['z'].mean(),
    'Average Length': df['length'].mean(),
    'Average Width': df['width'].mean(),
    'Average Height': df['height'].mean(),
    'PCA Components for 95% Variance': np.argmax(cumulative_variance >= 0.95) + 1,
    'Optimal K-Means Clusters': optimal_k,
    'DBSCAN Clusters': n_clusters,
    'DBSCAN Noise Points': n_noise
}

print("\n=== Dataset Summary ===")
for key, value in summary.items():
    if isinstance(value, float):
        print(f"{key}: {value:.2f}")
    else:
        print(f"{key}: {value}")

# Save summary to file
with open(os.path.join(cfg.LOG_DIR, 'data_exploration_summary.txt'), 'w') as f:
    f.write("=== Dataset Exploration Summary ===\n\n")
    for key, value in summary.items():
        if isinstance(value, float):
            f.write(f"{key}: {value:.2f}\n")
        else:
            f.write(f"{key}: {value}\n")

print("\nSummary saved to:", os.path.join(cfg.LOG_DIR, 'data_exploration_summary.txt'))